COLUMN_NAME EXPLANATION SAMPLE

drops 
PRIM_TKT_NUM: Müşteri bilet numarasıdır 2,307,664,176
CITY_NM: Varış istasyonunun şehir adıdır ISTANBUL
CNTRY_NM: Varış istasyonunun ülke adıdır TURKEY
ORG_CITY_NM: Kalkış istasyonunun şehir adıdır ADIYAMAN
ORG_CNTRY_NM: Kalkış istasyonunun ülke adıdır TURKEY

gropby:
CUST_KEY: Müşteriye özel bir numaradır 2670242

SEG_LCL_DEP_DT: GMT saat türünden kalkış günü bilgisidir 4/16/2017 0:00

ORGN_AP: Kalkış istasyonunun airport kodudur ADF
DSTN_AP: Varış istasyonunun airport kodudur IST

DOM_INTNL_FLAG: Seyahatin yurtiçi, yurtdışı olduğunu gösterir (D:Domestic, I: International)
JRNY_TYP: Seyahatin tek yön mü, gidiş- dönüş mü olduğunu gösterir (O:One-Way, R: Return)
TOP1_ORG_MKT_REGN: Müşterinin en çok çıkış yaptığı şehrin Pazarlama birimince  tanımlanan Region bilgisini içerir - Domestic
TOP1_ORG_CTY: Müşterinin en çok çıkış yaptığı şehirdir SIVAS

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
from collections import Counter

In [2]:
def find_transitions(df):
    
    flight_pairs = list(zip(df['ORG_CITY_NM'], df['CITY_NM']))
    pair_counts = Counter(flight_pairs)
    # all_cities = set(list(df['ORG_CITY_NM'].unique()) + list(df['CITY_NM'].unique()))
    shape = (len(all_cities_list), len(all_cities_list))
    
    transition_df = pd.DataFrame(np.zeros(shape),index=sorted(all_cities_list), columns=sorted(all_cities_list))
    
    for k, v in pair_counts.items():
        transition_df.loc[k[0], k[1]] = v
    
    return transition_df.div(transition_df.sum(axis=1)+0.0000001, axis=0)

In [3]:
def find_customer_transitions(cust_df):
    return find_transitions(cust_df)

In [4]:
def find_seasonal_transitions(df, season):
    seasonal_df = df[df['season']==season]
    return find_transitions(seasonal_df)

In [5]:
data_path = r'BirSonrakiDestinasyonTahminlemeBase.csv'

seperator = '|'
col_list = ['CUST_KEY', 'SEG_LCL_DEP_DT', 'ORG_CITY_NM', 'CITY_NM', 'DOM_INTNL_FLAG', 'JRNY_TYP',
        'TOP1_ORG_CTY']

dtypes = {'CUST_KEY': 'int32',
          'ORG_CITY_NM': 'category',
          'CITY_NM': 'category',
          'DOM_INTNL_FLAG': 'category',
          'JRNY_TYP': 'category',
          'TOP1_ORG_CTY': 'category'}
dates = ['SEG_LCL_DEP_DT']

## alternative cities

In [6]:
alt_cities_path = r'AlternatifSehirler_BirSonrakiDestinasyonTahminleme.xlsx'

replace_for_alter_cities = {'ordu_giresun':'ordu-giresun',
                            'dar_es_salaam':'dar es salaam',
                            'dalaman_mugla':'dalaman (mugla)',
                            'tel_aviv':'tel aviv',
                            'sharm_el_sheikh':'sharm el-sheikh',
                            'abu_dhabi':'abu dhabi',
                           'basel_mulhouse':'basel/mulhouse',
                           'amasya___merzifon':'amasya/merzifon',
                           'balikesir_edremit':'balikesir edremit'}

alter_cities = pd.read_excel(alt_cities_path, usecols=['City_1', 'City_2'])

for col in alter_cities.columns:
    alter_cities[col] = alter_cities[col].str.lower()
alter_cities.replace(replace_for_alter_cities, inplace=True)

In [7]:
df = pd.read_csv(data_path, usecols=col_list, sep=seperator, parse_dates=dates, encoding='cp437')

In [8]:
df.memory_usage().sum()/(1024*1024*1024)

6.014262229204178

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115317228 entries, 0 to 115317227
Data columns (total 7 columns):
 #   Column          Dtype         
---  ------          -----         
 0   CUST_KEY        int64         
 1   SEG_LCL_DEP_DT  datetime64[ns]
 2   CITY_NM         object        
 3   ORG_CITY_NM     object        
 4   DOM_INTNL_FLAG  object        
 5   JRNY_TYP        object        
 6   TOP1_ORG_CTY    object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 6.0+ GB


In [10]:
df.head(3)

,CUST_KEY,SEG_LCL_DEP_DT,CITY_NM,ORG_CITY_NM,DOM_INTNL_FLAG,JRNY_TYP,TOP1_ORG_CTY
0,2670242,2017-06-24,ISTANBUL,SIVAS,D,O,SIVAS
1,170285826,2019-05-21,TEL AVIV,MIAMI,I,R,MIAMI
2,171132805,2019-06-25,KUTAHYA,FRANKFURT,I,R,FRANKFURT


In [11]:
replace_for_errors= {'ADDS ABABA' : 'ADDIS ABABA',
                    'MOGADSHU' : 'MOGADISHU',
                    'SNOP' : 'SINOP',
                    'ABDJAN':'ABIDJAN',
                    'THESSALONK' : 'THESSALONIKI',
                    'SULEYMANAH' : 'SULEYMANIAH',
                    'HO CH MNH CTY (SAGON)' : 'HO CHI MINH CITY (SAIGON)',
                    'SHRAZ' : 'SHIRAZ',
                    'SHARM EL-SHEKH' : 'SHARM EL-SHEIKH',
                    'VALENCA' : 'VALENCIA',
                    'TURN(TORNO)' : 'TURIN(TORINO)',
                    'TEKRDAG' : 'TEKIRDAG',
                    'LEFKOA' : 'LEFKOSA',
                    'KLMANJARO' : 'KILIMANJARO',
                    'SRT' : 'SIIRT',
                    '╓STERSUND': 'OSTERSUND',
                     '─NGELHOLM': 'ANGELHOLM',
                     'MAL╔': 'MALI',
                     'TAF': 'TAIF',
                     'SEVLLE': 'SEVILLE',
                     'ASUNCI≤N': 'ASUNCION',
                     'NOVOSBRSK': 'NOVOSIBIRSK',
                     'OKNAWA': 'OKINAWA',
                     'LLONGWE': 'LILONGWE',
                     'MSURATA': 'MISURATA',
                     'VIT≤RIA': 'VITORIA',
                     'REYKJAVΦK': 'REYKJAVIK',
                     'SΠO LUΦS': 'SAO LUIS',
                     'OVEDO': 'OVIEDO',
                     'JOΠO PESSOA': 'JOAO PESSOA',
                     'CHAPEC≤': 'CHAPECO',
                     'SΠO JOSΘ DO RIO PRETO': 'SAO JOSE DO RIO PRETO',
                     'SAN JOSΘ DEL CABO': 'SAN JOSE DEL CABO',
                     'SAN PEDRO SULA\t': 'SAN PEDRO SULA',                     
                     'MAZAR-E SHARF': 'MAZAR-E SHARIF',
                     'SAN ANDRΘS': 'SAN ANDROS',
                     'TAPA': 'TAIPA',
                     'SPLT':'SPLIT',
                     'HAGΣT±A, GUAM': 'HAGATNA, GUAM',
                     'LINK÷PING': 'LINKOPING',
                     'C·CUTA': 'CUCUTA',
                     'ILHΘUS': 'ILHEUS',
                    'ABÃÂ°DJAN':'ABIDJAN',
                     'AB�DJAN':'ABIDJAN',
                     '─NGELHOLM': 'ANGELHOLM',
                    'ADDÃÂ°S ABABA' : 'ADDIS ABABA',
                     'ADD�S ABABA' : 'ADDIS ABABA',
                     'ASUNCIÃÂ³N': 'ASUNCION',
                     'CHAPEC≤': 'CHAPECO',
                     'CÃÂºCUTA': 'CUCUTA',
                     'HO CHÃÂ° MÃÂ°NH CÃÂ°TY (SAÃÂ°GO' : 'HO CHI MINH CITY (SAIGON)',
                     'HO CH� M�NH C�TY (SA�GON)' : 'HO CHI MINH CITY (SAIGON)',
                     'HAGΣT±A, GUAM': 'HAGATNA, GUAM',
                     'ILHΘUS': 'ILHEUS',
                     'LEFKOÃÅ¾A' : 'LEFKOSA',
                     'LEFKO�A':'LEFKOSA',
                     'LÃÂ°LONGWE': 'LILONGWE',
                     'L�LONGWE': 'LILONGWE',
                     'LINK÷PING': 'LINKOPING',
                    'KÃÂ°LÃÂ°MANJARO' : 'KILIMANJARO',
                     'K�L�MANJARO': 'KILIMANJARO',
                    'MOGADÃÂ°SHU' : 'MOGADISHU',
                     'MOGAD�SHU' : 'MOGADISHU',
                     'JOÃÂ£O PESSOA': 'JOAO PESSOA',
                     'MALÃÂ°': 'MALI',
                     'MÃÂ°SURATA': 'MISURATA',
                     'M�SURATA': 'MISURATA',
                     'MAZAR-E SHARÃÂ°F': 'MAZAR-E SHARIF',
                     'MAZAR-E SHAR�F': 'MAZAR-E SHARIF',
                     'NOVOSÃÂ°BÃÂ°RSK': 'NOVOSIBIRSK',
                     'NOVOS�B�RSK': 'NOVOSIBIRSK',
                     'OVÃÂ°EDO': 'OVIEDO',
                     'OV�EDO': 'OVIEDO',
                     'OKÃÂ°NAWA': 'OKINAWA',
                     'OK�NAWA': 'OKINAWA',
                     'ÃÂ�STERSUND': 'OSTERSUND',
                     'REYKJAVÃÂ­K': 'REYKJAVIK',
                    'SÃÂ°NOP' : 'SINOP',
                     'S�NOP': 'SINOP',
                    'SULEYMANÃÂ°AH' : 'SULEYMANIAH',
                     'SULEYMAN�AH': 'SULEYMANIAH',
                    'SHÃÂ°RAZ' : 'SHIRAZ',
                     'SH�RAZ': 'SHIRAZ',
                    'SHARM EL-SHEÃÂ°KH' : 'SHARM EL-SHEIKH',
                     'SHARM EL-SHE�KH': 'SHARM EL-SHEIKH',
                     'SÃÂ£O LUÃÂ­S': 'SAO LUIS',
                     'SEVÃÂ°LLE': 'SEVILLE',
                     'SEV�LLE': 'SEVILLE',
                     'SPLÃÂ°T':'SPLIT',
                     'SPL�T':'SPLIT',
                     'SÃÂ°ÃÂ°RT' : 'SIIRT',
                     'S��RT':'SIIRT',
                     'SΠO JOSΘ DO RIO PRETO': 'SAO JOSE DO RIO PRETO',
                     'SAN JOSÃÂ© DEL CABO': 'SAN JOSE DEL CABO',
                     'SAN PEDRO SULA\t': 'SAN PEDRO SULA',                     
                     'SAN ANDRΘS': 'SAN ANDROS',
                    'TURÃÂ°N(TORÃÂ°NO)' : 'TURIN(TORINO)',
                     'TUR�N(TOR�NO)': 'TURIN(TORINO)',
                    'TEKRDAG' : 'TEKIRDAG',
                     'TEK�RDAG': 'TEKIRDAG',
                     'TAÃÂ°F': 'TAIF',
                     'TA�F': 'TAIF',
                     'TAPA': 'TAIPA',
                     'UBERLÃÂ¢NDIA':'UBERLANDIA',
                    'THESSALONÃÂ°KÃÂ°' : 'THESSALONIKI',
                     'THESSALON�K�': 'THESSALONIKI',
                    'VALENCÃÂ°A' : 'VALENCIA',
                     'VALENC�A': 'VALENCIA',
                    'VITÃÂ³RIA': 'VITORIA'}

replace_for_errors = {k.lower(): v.lower() for k,v in replace_for_errors.items()}

In [12]:
for col in ['ORG_CITY_NM', 'CITY_NM', 'TOP1_ORG_CTY']:
    df[col] = df[col].str.lower()

In [13]:
import gc
gc.collect()

1622

In [14]:
for col in ['ORG_CITY_NM', 'CITY_NM', 'TOP1_ORG_CTY']:
    df[col] = df[col].replace(replace_for_errors)

In [15]:
# dropping flights that have same arrival and departure city
df = df[df['ORG_CITY_NM']!=df['CITY_NM']]

In [16]:
# # of customers that has more than 1 flight
sum(df['CUST_KEY'].duplicated(False))

94242881

In [17]:
df.shape

(114790271, 7)

## eda

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114790271 entries, 0 to 115317227
Data columns (total 7 columns):
 #   Column          Dtype         
---  ------          -----         
 0   CUST_KEY        int64         
 1   SEG_LCL_DEP_DT  datetime64[ns]
 2   CITY_NM         object        
 3   ORG_CITY_NM     object        
 4   DOM_INTNL_FLAG  object        
 5   JRNY_TYP        object        
 6   TOP1_ORG_CTY    object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 6.8+ GB


In [19]:
df.isna().sum()

CUST_KEY               0
SEG_LCL_DEP_DT         0
CITY_NM            51284
ORG_CITY_NM        39349
DOM_INTNL_FLAG      4791
JRNY_TYP           22935
TOP1_ORG_CTY      184129
dtype: int64

In [20]:
len(df['CUST_KEY'].unique())

36576424

In [21]:
%%script False
import missingno as msgn
#read in data here
msgn.matrix(df,color=(0.45, 0.75, 0.25))

Couldn't find program: 'False'


In [22]:
df.dropna(subset=['CITY_NM', 'ORG_CITY_NM', 'TOP1_ORG_CTY'],inplace=True)

In [23]:
seasons = {'winter':[12,1,2], 
           'spring':[3,4,5], 
           'summer':[6,7,8],
           'fall':[9,10,11]}

season_mapping = {month:k for k,v in seasons.items() for month in v}
season_mapping

{12: 'winter',
 1: 'winter',
 2: 'winter',
 3: 'spring',
 4: 'spring',
 5: 'spring',
 6: 'summer',
 7: 'summer',
 8: 'summer',
 9: 'fall',
 10: 'fall',
 11: 'fall'}

In [24]:
df['season'] = df['SEG_LCL_DEP_DT'].apply(lambda x: season_mapping[x.month])

In [25]:
df.isna().sum()

CUST_KEY             0
SEG_LCL_DEP_DT       0
CITY_NM              0
ORG_CITY_NM          0
DOM_INTNL_FLAG    2506
JRNY_TYP          4539
TOP1_ORG_CTY         0
season               0
dtype: int64

In [26]:
df.shape

(114541202, 8)

In [27]:
# converting columns to corresponding column types
for k, v in dtypes.items():
    df[k] = df[k].astype(v)

In [28]:
# finding flight pairs as dep-arr
flight_pairs = list(zip(df['ORG_CITY_NM'],df['CITY_NM']))
# counting flight pairs
pair_counts = Counter(flight_pairs)
# top 10 flight pairs
sorted(pair_counts.items(), key=lambda pair: pair[1], reverse=True)[0:10]

[(('istanbul', 'izmir'), 3321736),
 (('istanbul', 'ankara'), 3286059),
 (('ankara', 'istanbul'), 3065330),
 (('izmir', 'istanbul'), 2851972),
 (('istanbul', 'antalya'), 2434309),
 (('istanbul', 'adana'), 1944382),
 (('istanbul', 'trabzon'), 1877503),
 (('adana', 'istanbul'), 1743310),
 (('antalya', 'istanbul'), 1726738),
 (('trabzon', 'istanbul'), 1519597)]

In [74]:
path = r'BIR_SONRAKI_DEST_VALSET.csv'
cols = ['CUST_KEY','CITY_NM', 'ORG_CITY_NM', 'TOP1_ORG_CTY']
val = pd.read_csv(path, sep='|', usecols=cols)
val.dropna(subset=['CITY_NM', 'ORG_CITY_NM', 'TOP1_ORG_CTY'],inplace=True)
for col in ['ORG_CITY_NM', 'CITY_NM', 'TOP1_ORG_CTY']:
    val[col] = val[col].str.lower()
val.replace(replace_for_errors, inplace=True)

In [30]:
# finding all unique cities
all_cities_list = set(list(df['ORG_CITY_NM'].unique()) + list(df['CITY_NM'].unique())+list(val['ORG_CITY_NM'].unique())+list(val['CITY_NM'].unique())+list(val['TOP1_ORG_CTY'].unique()))

ax = sns.countplot(x="season", data=df)

## splitting data as train and test

In [31]:
%%script False
# grouping customers to find latest flights of each customer to split as test data
grouped_df = df.sort_values(by=['CUST_KEY', 'SEG_LCL_DEP_DT']).groupby(by='CUST_KEY')
test_indexes = grouped_df['SEG_LCL_DEP_DT'].idxmax()
with open("test_indexes.pickle","wb") as file:
    pickle.dump(test_indexes,file)

Couldn't find program: 'False'


In [32]:
%%script False
with open("test_indexes.pickle",'rb') as file:
    test_indexes = pickle.load(file)

Couldn't find program: 'False'


In [33]:
# customers who has 1 flights used as test

# flights of the customers who have only one flight
test_df1 = df.drop_duplicates(subset=['CUST_KEY'], keep=False)

df = df[df['CUST_KEY'].duplicated(False)]

# last flights of the customers who have more than one flight
# test_df2 = df.loc[test_indexes]

# train_df = df.drop(test_indexes)

In [34]:
df.head()

,CUST_KEY,SEG_LCL_DEP_DT,CITY_NM,ORG_CITY_NM,DOM_INTNL_FLAG,JRNY_TYP,TOP1_ORG_CTY,season
0,2670242,2017-06-24,istanbul,sivas,D,O,sivas,summer
3,107697983,2019-07-26,antalya,moscow,I,R,moscow,summer
5,22455129,2016-12-12,istanbul,adiyaman,D,O,adiyaman,winter
6,111468676,2019-02-14,izmir,ankara,D,O,izmir,winter
7,7242909,2019-12-06,istanbul,van,D,O,van,winter


In [35]:
%%script False
# use this for graph
count_by_days = df.groupby(by='SEG_LCL_DEP_DT').count()
ax = count_by_days.plot( y='CUST_KEY', figsize=(12,6))

Couldn't find program: 'False'


In [36]:
all_cities_list

{'saipan island',
 'santa marta',
 'porto alegre',
 'vitoria da conquista',
 'la paz',
 'iloilo',
 'belgrade',
 'hakkari',
 'kavala',
 'bari',
 'najaf',
 'bandar lampung',
 'freeport',
 'horta',
 'yiwu',
 'fortaleza',
 'malatya',
 'malé',
 'varadero',
 'poza rica',
 'menorca',
 'shiraz',
 'rodes island',
 'portland',
 'tahiti',
 'abakan',
 'ibiza',
 'amsterdam',
 'datong',
 'catania',
 'bandar seri begawan',
 'chicago',
 'reynosa',
 'gainesville',
 'mostar',
 'pasco',
 'mobile',
 'adana',
 'kittila',
 'baotou',
 'willemstad',
 'khanty-mansiysk',
 'montgomery',
 'williams lake',
 'bacau',
 'clermont-ferrand',
 'peshawar',
 'tottori',
 'norilsk',
 'noumea',
 'visakhapatnam',
 'vijayawada',
 'frankfurt',
 'mazatlan',
 'hayden',
 'fort st.john',
 'dammam',
 'palma de mallorca',
 'armenia',
 'floro',
 'ioannina',
 'puerto iguazu',
 'redding',
 'nottingham',
 'kupang',
 'rostock',
 'mykonos island',
 'dusseldorf',
 'fort myers',
 'kimberley',
 'almeria',
 'pune',
 'ängelholm',
 'madurai',
 '

In [37]:
general_transitions = find_transitions(df)
seasonal_transitions_summer = find_seasonal_transitions(df, 'summer')
seasonal_transitions_winter = find_seasonal_transitions(df, 'winter')
seasonal_transitions_spring = find_seasonal_transitions(df, 'spring')
seasonal_transitions_fall = find_seasonal_transitions(df, 'fall')

seasonal_transitions = {'summer':seasonal_transitions_summer,
                        'winter':seasonal_transitions_winter,
                        'spring':seasonal_transitions_spring,
                        'fall':seasonal_transitions_fall}

In [38]:
import pickle

In [39]:
df_pickle = df[['CUST_KEY','ORG_CITY_NM', 'CITY_NM', 'season', 'TOP1_ORG_CTY']].copy()

In [40]:
df_pickle['season'] = df_pickle['season'].astype('category')

In [41]:
df_pickle.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94165087 entries, 0 to 115317227
Data columns (total 5 columns):
 #   Column        Dtype   
---  ------        -----   
 0   CUST_KEY      int32   
 1   ORG_CITY_NM   category
 2   CITY_NM       category
 3   season        category
 4   TOP1_ORG_CTY  category
dtypes: category(4), int32(1)
memory usage: 1.7 GB


In [42]:
with open("train_df.p","wb") as f:
    pickle.dump(df_pickle, f)

In [78]:
with open("general_transitions.p","wb") as f:
    pickle.dump(general_transitions, f)
with open("seasonal_transitions.p","wb") as f:
    pickle.dump(seasonal_transitions, f)
with open("train_df.p","wb") as f:
    pickle.dump(df, f)